In [6]:
from tagmatch.vec_db import Embedder
import numpy as np
from tagmatch.vec_db import VecDB
import pickle
from sentence_transformers.quantization import quantize_embeddings
from create_dummy_tags import tags
from pydantic_settings import BaseSettings
class Settings(BaseSettings):
    model_name: str
    cache_dir: str
    qdrant_host: str
    qdrant_port: int
    qdrant_collection: str
    reduced_embed_dim: int

    class Config:
        env_file = ".env"
settings=Settings()

/opt/homebrew/Caskroom/miniconda/base/envs/tagsearch/lib/python3.11/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ('settings_',)`.
  warnings.warn(


In [4]:
embedder = Embedder(model_name=settings.model_name,cache_dir=settings.cache_dir)
tag_embeddings = [embedder.embed(tag) for tag in extended_tags]
embed_vecs=np.asarray(tag_embeddings)

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 33825.03it/s]


In [35]:
qembed_vecs = quantize_embeddings(embed_vecs,precision="binary")

In [37]:
vec_db = VecDB(
    host="http://localhost",
    port=6333,
    collection="pca_train_test_reduced",
    vector_size=48,
)

In [38]:
for name,vec in zip(tags,qembed_vecs):
    vec_db.store(vec,{"name":name})

In [40]:
reduced_vector_matches = [vec_db.find_closest(query_vector, 5) for query_vector in qembed_vecs]

In [42]:
reduced_vector_matches[0]

[ScoredPoint(id=9682909050782155870, version=0, score=1.0, payload={'name': 'Apollo 11'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=11658385836130133823, version=109, score=0.460904, payload={'name': 'Planetary Protection'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=12092732572154152545, version=139, score=0.440875, payload={'name': 'Space Industry'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=13348738339898441951, version=55, score=0.41671443, payload={'name': 'Colonization'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=9984194481627360211, version=14, score=0.41425395, payload={'name': 'NASA'}, vector=None, shard_key=None, order_value=None)]

In [44]:
vec_db_full = VecDB(
    host="http://localhost",
    port=6333,
    collection="train_test",
    vector_size=embedder.embedding_dim,
)

In [45]:
for name,vec in zip(tags,embed_vecs):
    vec_db_full.store(vec,{"name":name})

In [46]:
vector_matches = [vec_db_full.find_closest(query_vector, 5) for query_vector in embed_vecs]

In [47]:
vector_matches[0]

[ScoredPoint(id=13374045570609575028, version=0, score=0.9999999, payload={'name': 'Apollo 11'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=12127991403884983481, version=46, score=0.7471009, payload={'name': 'Moon Mission'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=10662690989898557273, version=1, score=0.72500336, payload={'name': 'Moon'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=12915431704230016658, version=80, score=0.7157799, payload={'name': 'Lunar Base'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=13405533713018267398, version=14, score=0.6981933, payload={'name': 'NASA'}, vector=None, shard_key=None, order_value=None)]